# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [2]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2022-04-15 19:01:24--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt [following]
--2022-04-15 19:01:25--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  3.57MB/s    in 1.3s    

2022-04-15 19:01:27 (3.57 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [3]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [4]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('york', 0.7871513962745667),
 ('queen', 0.7778061628341675),
 ('prince', 0.7777647376060486),
 ('clarence', 0.7662815451622009),
 ('duke', 0.7593518495559692),
 ('son', 0.728421688079834),
 ('gloucester', 0.7273623943328857),
 ('princess', 0.719609797000885),
 ('cardinal', 0.7134799957275391),
 ('northumberland', 0.7034063339233398)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [6]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9615752696990967),
 ('iago', 0.9489233493804932),
 ('ham', 0.9299837350845337),
 ('lucio', 0.9185928702354431),
 ('emilia', 0.9130783677101135),
 ('cassio', 0.9054528474807739),
 ('cressida', 0.8995676636695862),
 ('glou', 0.8972945213317871),
 ('proteus', 0.897049069404602),
 ('mrs', 0.895332396030426)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [7]:
model.wv.similarity('othello', 'desdemona')

0.9615754

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [8]:
## TODO: Implement cosim

from numpy import dot 
from numpy.linalg import norm 
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  result = dot(v1, v2)/(norm(v1) * norm(v2))
  return result

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.96157545

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [10]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2022-04-15 19:05:51--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec [following]
--2022-04-15 19:05:52--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M  14.6MB/s    in 5.6s    

2022-04-15 19:05:58 (11.5 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2022-04-15 19:05:59--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [11]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [16]:
## TODO: implement this search function

def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0
  l = {}
  for i in vecDict:
    a = cosim(vec, vecDict[i])
    l[i] = a
  mostSimilar = max(l, key=l.get)
  return (mostSimilar, l[mostSimilar])

## some example searches

[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [18]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2022-04-15 19:15:46--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab [following]
--2022-04-15 19:15:47--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  1.49MB/s    in 0.9s    

2022-04-15 19:15:48 (1.49 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [19]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.

accuracy = 0
baselineAccuracy = 0
l3 = 0
for j in links:
  b = envec[j[0]]
  mostsimi = mostSimilar(b, frvec)
  if mostsimi[0] == j[2]:
    l3 += 1 
print(l3)

#Compute baseline accuracy

l1 = 0
l2 = 0
for i in links:
  if i[1] =='fr':
    if i[0] == i[2]:
      l2 += 1
    l1 += 1
baselineAccuracy = l2/l1

**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-X Wikipedia retrieval accuracy.

accuracy = 0
baselineAccuracy = 0

l3 = 0
for j in links:
  b = envec[j[0]]
  mostsimi = mostSimilar(b, devec)
  if mostsimi[0] == j[2]:
    l3 += 1 
print(l3)

#Compute baseline accuracy

l1 = 0
l2 = 0
for i in links:
  if i[1] =='de':
    if i[0] == i[2]:
      l2 += 1
    l1 += 1
baselineAccuracy = l2/l1

In [ ]:
import operator 
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse = True )


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [ ]:
import operator
def tenmostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0
  l = {}
  for i in vecDict:
    a = cosim(vec, vecDict[i])
    l[i] = a
  sorted_x = sorted(l.items(), key=operator.itemgetter(1), reverse=True)
  return sorted_x[:10]

In [ ]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.

for a, b, c in links:
  listofsimilars = tenmostSimilar
  count = 0 
  for i in range(len(listofsimilars)):
    if listofsimilars[i] == c:
      count += 1 
      MRR += 1/(i+1)
recall = count/len(links)
MRR = MRR/len(links)

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.